In [5]:
import requests
from requests import exceptions
from bs4 import BeautifulSoup
import urllib3
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [9]:
urllib3.disable_warnings()

In [24]:
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome('C:\\Users\\Shahad\\Downloads\\chromedriver_win32\\chromedriver.exe')
    #options.headless = True
    #driver = webdriver.Chrome('C:\\Users\\Shahad\\Downloads\\chromedriver_win32\\chromedriver.exe', options=options)  
    #driver.maximize_window()
    driver.get('https://conti.plus/en/overview')

In [25]:
driver.get('https://conti.plus/en/product/detail/id/272')
driver.page_source

'<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en"><head>\n    <meta name="keywords" content="CONTI WV-series touch-less faucet tap water use reduction">\n<meta name="description" content="lino concealed wall-mounted lavatory faucet WV-series, IR-sensor, chrome, 12-, 24- or 48-hour hygienic rinse activatable with click, follow-up time and sensor sensitivity adjustable with click">\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\n<link rel="stylesheet" type="text/css" href="/assets/58d02e0d/jquery.fancybox-1.3.4.css">\n<link rel="stylesheet" type="text/css" href="/assets/50c2b74e/cycle2.css">\n<link rel="stylesheet" type="text/css" href="/assets/987c7962/css/bootstrap.min.css">\n<link rel="stylesheet" type="text/css" href="/assets/987c7962/css/bootstrap-responsive.min.css">\n<link rel="stylesheet" type="text/css" href="/assets/987c7962/css/yiistrap.min.css">\n<link rel="stylesheet" type="text/css" href="/assets/50c2b74e/bootstrap-hightech.css">\n

In [6]:
def create_soup(url):
    try:
        r = requests.get(url, verify=False)
        soup = BeautifulSoup(r.text, 'lxml')
        return soup
    except:
        return 'N/A'

In [14]:
soup = create_soup('https://conti.plus/en/product/detail/id/272')

In [11]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="CONTI WV-series touch-less faucet tap water use reduction" name="keywords"/>
<meta content="lino concealed wall-mounted lavatory faucet WV-series, IR-sensor, chrome, 12-, 24- or 48-hour hygienic rinse activatable with click, follow-up time and sensor sensitivity adjustable with click" name="description"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="/assets/58d02e0d/jquery.fancybox-1.3.4.css" rel="stylesheet" type="text/css"/>
<link href="/assets/50c2b74e/cycle2.css" rel="stylesheet" type="text/css"/>
<link href="/assets/987c7962/css/bootstrap.min.css" rel="stylesheet" type="text/css"/>
<link href="/assets/987c7962/css/bootstrap-responsive.min.css" rel="stylesheet" type="text/css"/>
<link href="/assets/987c7962/css/yiistrap.min.css" rel="s

In [47]:
def page_driver(item_id):
    
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome('C:\\Users\\Shahad\\Downloads\\chromedriver_win32\\chromedriver.exe')
    #options.headless = True
    #driver = webdriver.Chrome('C:\\Users\\Shahad\\Downloads\\chromedriver_win32\\chromedriver.exe', options=options)  
    driver.maximize_window()
    driver.get('https://conti.plus/en/overview')    
    try:
        popup_btn = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CLASS_NAME, 'consent-give'))
        )
        popup_btn.click()


        search_icon = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, r"//ul[@class='pull-right nav']/li/a"))
        )
        search_icon.click()


        search_bar = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CLASS_NAME, 'input-xlarge'))
        )
        search_bar.send_keys(item_id)


        search_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME, 'yt0'))
        )
        search_btn.click()

        select_item = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, item_id))
        )
        select_item.click()

        url = driver.current_url
        return url

    except:
        pass

In [45]:
df = pd.read_excel(r'Canti_Plus_links.xlsx')

In [44]:
def getURLs():
    url_lst = []
    for i in df.index:
        item_url = page_driver(str(df['Item Code '][i]))
        url_lst.append(item_url)
    return url_lst

In [48]:
urls = getURLs()
df['webpage_to_be_scraped'] = urls

In [205]:
def get_item_name(soup):
    try:
        name = soup.find('span', {'class' : 'span6'}).find('h1').text
        return name
    except:
        pass

In [227]:
def get_item_description(soup):
    try:
        s_par = ''
        paragraphs = soup.find('div', {'class' : 'tab-pane fade active in'}).findAll('p')
        for p in paragraphs:
            par = p.text.strip()
            s_par = s_par + par
        return s_par
    except:
        pass

In [231]:
def get_product_info_table(soup):
    try:
        body = soup.find('table', {'class' : 'table table-striped table-bordered'}).findAll('tr')
        inner_dictionary = {}

        for row in body:
            cols = row.children
            cols = [ele.text.strip() for ele in cols]
            inner_dictionary.update({cols[0] : cols[1:3]})
        return inner_dictionary
    except:
        pass

In [200]:
def get_specs(soup):
    try:
        spec = soup.find('div', {'id' : 'tab_2'}).find('p')
        spec = spec.get_text(strip=True, separator="\n")
        spec = spec.split('\n')
        outer_dictionary= {}
        for i in spec:
            #print(i)
            lst = i.split(':')
            #print(lst)
            inner_dictionary = {lst[0] : lst[1]}
            outer_dictionary.update(inner_dictionary)
        return outer_dictionary
    except:
        pass

In [241]:
def get_spare_parts_img(soup):
    try:
        img = soup.find('div', {'id' : 'tab_4'}).find('img').get('src')
        img = 'https://conti.plus' + img
        return img
    except:
        pass

In [202]:
def get_spare_parts_info(soup):
    try:
        outer_dictionary = {}
        col_1 = [] 
        col_2 = [] 
        col_3 = [] 
        
        dic_1 = {}
        dic_2 = {}
        dic_3 = {}
        counter = 0
        header = soup.find('div', {'id' : 'tab_4'}).find('table').findAll('th')
        header.remove(header[0])
        rows = soup.find('div', {'id' : 'tab_4'}).find('table').find('tbody').findAll('tr')
        
        for h in header:
            th = h.text.strip()
            counter = counter + 1
            if counter == 1:
                for r in rows:
                    col_1.append(r.find('td').find_next('td').text.strip())
                    r.find_next('tr')
                dic_1.update({ th : col_1 })
                outer_dictionary.update(dic_1)
            if counter == 2:
                for r in rows:
                    col_2.append(r.find('td').find_next('td').find_next('td').text.strip())
                    r.find_next('tr')
                dic_2.update({ th : col_2 })
                outer_dictionary.update(dic_2)
            if counter == 3:
                for r in rows:
                    col_3.append(r.find('td').find_next('td').find_next('td').find_next('td').text.strip())
                    r.find_next('tr')
                dic_3.update({ th : col_3 })
                outer_dictionary.update(dic_3)
        return outer_dictionary
    except:
        pass

In [253]:
def get_product_files(soup):
    try:
        img_dic = {}
        tr = soup.find('div', {'id' : 'tab_6'}).find('table').findAll('tr')
        for i in tr:
            img = i.find('a').get('href')
            img = 'https://conti.plus' + img
            title = i.find('a').text
            img_dic.update({ title : img })
        return img_dic
    except:
        pass

In [39]:
def get_product_img(soup):
    try:
        images = soup.find('div', {'class' : 'span3'}).find('div').findAll('img')
        img_dic = {}
        num = 1
        for i in images:
            img = 'https://conti.plus' + i.get('src')
            img_dic.update({ 'Image_'+str(num) : img })
            num = num + 1
        return img_dic
    except:
        pass

In [40]:
get_product_img(soup)

{'Image_1': 'https://conti.plus/media/conti2016/produktgruppen/image/WV10_800x533.jpg',
 'Image_2': 'https://conti.plus/media/conti2016/produktgruppen/image/DIMM_WV10_72dpi_RGB.jpg',
 'Image_3': 'https://conti.plus/media/conti2016/produktgruppen/image/DimmAP_WV10_800x533_1.jpg',
 'Image_4': 'https://conti.plus/media/conti2016/produktgruppen/image/WV10_AmbienteKuche_800x533.jpg'}

In [46]:
df['soup'] = df['webpage_to_be_scraped'].apply(lambda x: create_soup(x))

In [206]:
df['item name'] = df['soup'].apply(lambda x: get_item_name(x))

In [229]:
df['item description'] = df['soup'].apply(lambda x: get_item_description(x))

In [232]:
df['get_product_info_table'] = df['soup'].apply(lambda x: get_product_info_table(x))

In [234]:
df_temp = df['get_product_info_table'].apply(pd.Series)

In [236]:
df = pd.concat([df, df_temp], axis=1)

In [237]:
df['specs'] = df['soup'].apply(lambda x: get_specs(x))

In [239]:
df_temp = df['specs'].apply(pd.Series)

In [240]:
df = pd.concat([df, df_temp], axis=1)

In [242]:
df['get_spare_parts_img'] = df['soup'].apply(lambda x: get_spare_parts_img(x))

In [244]:
df['get_spare_parts_info'] = df['soup'].apply(lambda x: get_spare_parts_info(x))

In [246]:
df_temp = df['get_spare_parts_info'].apply(pd.Series)

In [247]:
df = pd.concat([df, df_temp], axis=1)

In [255]:
df['get_product_files'] = df['soup'].apply(lambda x: get_product_files(x))

In [256]:
df_temp = df['get_product_files'].apply(pd.Series)

In [251]:
df = pd.concat([df, df_temp], axis=1)

In [48]:
df['get_product_img'] = df['soup'].apply(lambda x: get_product_img(x))

In [50]:
df_temp = df['get_product_img'].apply(pd.Series)

In [51]:
df = pd.concat([df, df_temp], axis=1)

In [53]:
df.to_excel('Canti_Plus_complete.xlsx')